# Data dependence on g, spectral amplitude (flux) varied

In [9]:
import numpy as np
import random
import time

In [10]:
import sys
import os

In [ ]:
import swyft
import torch
from pytorch_lightning.loggers import WandbLogger
import wandb

In [11]:
import matplotlib
import matplotlib.pyplot as plt

In [12]:
import IPython

In [13]:
import pickle

In [14]:
%matplotlib inline

In [15]:
files_name = "spectral_params_and_B_config" + "_files"
parent_dir = (os.getcwd()+"/").split('/notebooks/', 1)[0]
start_dir = os.getcwd()
files_dir = os.getcwd() + "/" + files_name 
store_name = files_name.split("_files")[0] + "_store"
store_dir = os.getcwd() + "/" + files_name + "/" + store_name
sys.path.append(files_dir)

In [16]:
from ALP_quick_sim import ALP_sim

In [17]:
with open(files_dir+"/"+'config_objects.pickle', 'rb') as file:
    config_objects = pickle.load(file)
    
for key in config_objects.keys():
    locals()[key] = config_objects[key]

with open(files_dir+"/"+'obs_objects.pickle', 'rb') as file:
    obs_objects = pickle.load(file)
    
for key in obs_objects.keys():
    locals()[key] = obs_objects[key]

# Load simulations again

In [ ]:
store = swyft.ZarrStore(store_dir + "/" + store_name)
samples = store.get_sample_store()

In [ ]:
len(samples)

In [ ]:
A.configure_plot(xmin=6e1, xmax=1e3, ymin=-0.15,ymax=0.05,legend=0, logx=1, logy=0)
for i in range(min(len(samples),100)):
    A.configure_model(params=samples[i]['params'])
    A.import_counts(obs=samples[i]['data'],exp=samples[i]['exp'])
    new_fig = 1 if i==0 else 0
    A.compute_case(new_fig=new_fig,new_counts=0,color=None,color_obs=None,linestyle="-",
                   label_exp=0,label_obs=False, errorbands=False, errors=False, plot_exp=0, transparency=0.7)


for i,obs in enumerate(observations):
    A.configure_model(params=obs['params'])
    A.import_counts(obs=obs['data'],exp=obs['exp'])
    shade = i/(len(observations)+1)
    A.compute_case(new_fig=0,new_counts=0,color='k',color_obs=colors[i],linestyle_obs="-",label_exp=0,label_obs=0,
                   errorbands=0, errors=0, plot_exp=0)
    A.compute_case(new_fig=0,new_counts=0,color=colors[i],color_obs=colors[i],linestyle=":",label_exp=0,label_obs=0,
                        errorbands=0, errors=0, plot_obs=0) 

# Train again

In [1]:
class Network(swyft.SwyftModule):
    def __init__(self):
        super().__init__()
        self.norm = swyft.networks.OnlineStandardizingLayer(torch.Size([A.nbins]), epsilon=0)
        self.logratios = swyft.LogRatioEstimator_1dim(
            num_features = A.nbins, 
            num_params = len(marginals), 
            varnames = sim.A.param_names)
        self.learning_rate = 0.0005
    
    def forward(self, A, B):
        data = self.norm(A['data'])
        return self.logratios(data, B['params'][:,marginals])

NameError: name 'swyft' is not defined

In [ ]:
network = Network()

In [ ]:
wandb_logger = WandbLogger(log_model='all')

In [ ]:
trainer = swyft.SwyftTrainer(
    accelerator = 'cpu', precision = 64, logger=wandb_logger #, min_epochs =25, 
)

In [ ]:
dm = swyft.SwyftDataModule(samples)

In [ ]:
trainer.fit(network, dm)

In [ ]:
wandb.finish()

# Infer again

In [ ]:
prior_samples = sim.sample(100_000, targets=['params'])

In [ ]:
for j in range(len(marginals)):
    logratios = trainer.infer(
                            network,
                            observations[j],
                            prior_samples
                            )
    
    swyft.plot_posterior(logratios, A.param_names, truth={A.param_names[i]:truths[j][i] for i in range(len(truths[j]))})